# Quiz 5, Big Data 
Author: Ramtin Hosseini   

In [1]:
!pip install google-cloud-storage

In [2]:
%set_env GOOGLE_APPLICATION_CREDENTIALS=/Users/ramtin/Desktop/big_data_hw5/comp119-2a94715b87f1.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/ramtin/Desktop/big_data_hw5/comp119-2a94715b87f1.json


In [3]:
import json
import tweepy
from google.cloud import storage
storage_client = storage.Client()
bucket = storage_client.get_bucket("ramtin_comp119")
blob = storage.blob.Blob("tweeter_credential.json", bucket)
creds_bytes = blob.download_as_string()
_creds = json.loads(str(creds_bytes, 'utf-8'))
auth = tweepy.OAuthHandler(_creds["consumer_key"], _creds["consumer_secret"])
auth.set_access_token(_creds["access_token"], _creds["access_token_secret"])
del _creds # delete _creds immediately!
API = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
def show_tweet(tweet):
    try:
        return ' '.join([tweet.created_at.strftime("%m/%d/%Y, %H:%M:%S"), tweet.user.screen_name,tweet.text])

    except:
        return '************************ bad or no tweet ************************'

In [5]:
public_tweets = API.home_timeline()
for tweet in public_tweets[:1]:
    print(show_tweet(tweet))

11/25/2019, 16:30:28 Banisadr تجربه تاریخ میگوید: قدرت وقتی میتواند جنبشی را سرکوب کند که آنرا در مدار بسته خشونت گرفتار ‌سازد.
جنبش آبان هم همین… https://t.co/AvZuw7ZOlu


In [6]:
nyt = API.get_user('@nytimes')

In [7]:
# Tweets from your prominent user
user = "@nytimes" 
user_tweets = API.user_timeline(user)
for tweet in user_tweets[:3]:
    print(show_tweet(tweet))

12/03/2019, 01:00:24 nytimes Here's your guide to finding a genuinely great movie to watch on Netflix https://t.co/B6BUqWdO3a
12/03/2019, 00:40:05 nytimes RT @sheeraf: Think your Thanksgiving was rough? Imagine having to explain to family members that your company isn’t responsible for destroy…
12/03/2019, 00:20:06 nytimes Evening Briefing: Here's what you need to know at the end of the day https://t.co/ojK3xJKwSp


In [8]:
 # Some followers of your favorite user
user_friends = API.friends(user)
tweeters = [nyt.id_str] + [friend.id_str for friend in user_friends] # tweeters

In [9]:
print([friend.screen_name for friend in user_friends])

['davidenrich', 'marywalshnyt', 'collinskeith', 'johnnydiaz__', 'timesrunclub', 'cflav', 'aimee_ortiz', 'S_Evangelina', 'ShaneGoldmacher', 'kashhill', 'antontroian', 'kylebuchanan', 'NYTSports', 'ElianPeltier', 'eringriffith', 'FlitterOnFraud', 'gillianwong', 'Nataliekitro', 'caitlinroper', 'AlanYuhas']


In [10]:
import time
import tweepy
ids = []
for index, page in enumerate(tweepy.Cursor(API.followers_ids, screen_name='nytimes').pages()):
    ids.extend(page)
    if len(ids)>= 10:
        break

In [13]:
# tweeters = [nyt.id_str] + [friend.id_str for friend in user_friends]

In [14]:
ids_str = []
for each in ids:
    ids_str.append(str(each))

print(ids_str)

['1079022933035626497', '1145487013715382272', '1201651901370163200', '1193100875507621888', '1194119634544259073', '1201668342219558914', '2354777088', '1201668568909262848', '1681735532', '1183907070577389569', '1201665580719230976', '1109116946', '3502470501', '1102660042803818497', '1086997984288620544', '1196662169749934081', '794418831141703680', '2956204483', '1060293960134987777', '1144911555726733312', '714883880990351360', '1128230965099155458', '2703350377', '1201667360106668032', '1133441535368994819', '1201593135656251399', '1201306268561883136', '1117240223400955904', '1201667136797708288', '1148076118991200257', '1201659251166498817', '1201631809714282496', '1201619689723629568', '948886285124501504', '1199409821528797185', '624319513', '995049197245169664', '1200897688344113158', '1171405831457271808', '1183445298321883137', '1117649600', '1201667371183857664', '832577215762010115', '1182143688543232000', '1201666958682284032', '1123456030866845696', '120166671663330508

In [15]:
import numpy as np
import json
with open("index.json") as json_file:
    afinn_dict = json.load(json_file)

def get_average_sentiment_score(a_tweet):
    if len(a_tweet) ==0:
        return 0
    else:
        list_words_in_a_tweet = a_tweet.split(' ')
        list_sentiment_score = []
        counter = 1
        for a_word in list_words_in_a_tweet:
            sentiment_score = afinn_dict.get(a_word, 0)
            list_sentiment_score.append(sentiment_score)
            if sentiment_score !=0:
                counter+=1
        sum_sentiment_a_tweet = np.sum(list_sentiment_score)
        avg_sentiment_a_tweet = sum_sentiment_a_tweet/counter
        return avg_sentiment_a_tweet
     

In [16]:
def create_dict_from_hashtags(tweets_list):
    dict_hastags = {}
    for a_tweet in tweets_list:
        elements_in_a_tweet = a_tweet.split( )
        for an_element in elements_in_a_tweet:
            if an_element.startswith('#'):
                dict_hastags[an_element] = dict_hastags.setdefault(an_element, 0) + 1
                
    return dict_hastags           


In [17]:
def find_the_most_popular_hashtag(dict_hashtags):
    if len(dict_hashtags)>0:
        return max(dict_hashtags, key=dict_hashtags.get)
    else:
        return 'none'

In [18]:
def return_a_list_tweets_with_max_hasthags(tweets_list, most_popular_hashtag):
    list_tweets_with_max_hashtag = []
    for a_tweet in tweets_list:
        elements_in_a_tweet = a_tweet.split( )
        for an_element in elements_in_a_tweet:
            if an_element == most_popular_hashtag:
                list_tweets_with_max_hashtag.append(a_tweet)
    return(list_tweets_with_max_hashtag)

In [19]:
def return_a_list_tweets_with_this_hasthag(tweets_list, this_hashtag):
    list_tweets_with_this_hashtag = []
    for a_tweet in tweets_list:
        elements_in_a_tweet = a_tweet.split( )
        for an_element in elements_in_a_tweet:
            if an_element == this_hashtag:
                list_tweets_with_this_hashtag.append(a_tweet)
    return list_tweets_with_this_hashtag

In [38]:
def return_sign_changes_and_amount_change(x, y):
    epsilon = 1e-6
    sign = 'neutral'
    if (y - x) / (x + epsilon) >= 1/4: 
        sign = '+'
        return sign, (y - x)/(x + epsilon)
    elif (x - y) / (x + epsilon) >= 1/4:
        sign = '-'
        return sign, (x - y)/(x + epsilon)
    else:
        return sign, epsilon
        

In [21]:
import time
def return_tuple_regarding_sudden_chages_in_a_particular_hashtag(list_of_dict_hastag_sentiment):
    dict_in_ten_min = {}
    list_keys_in_ten_min = []
    for each_dict in list_of_dict_hastag_sentiment:
        list_keys_in_a_window = list(each_dict.keys())
        for each_key in list_keys_in_a_window:
            list_keys_in_ten_min.append(each_key)

    unique_keys_list_in_ten_min = list(set(list_keys_in_ten_min)) 

    for each in unique_keys_list_in_ten_min:
        dict_in_ten_min[each] = []    

    for each_dict in list_of_dict_hastag_sentiment:
        keys_in_each_dict = list(each_dict.keys())
        for key in keys_in_each_dict:
            dict_in_ten_min[key].append(each_dict[key])

    list_tuples_sudden_chages = []
    for each_key in dict_in_ten_min:
        sentiment_specifc_hashtag_in_ten_min = dict_in_ten_min[each_key]
        if len(sentiment_specifc_hashtag_in_ten_min)== 1:
            continue
        elif len(sentiment_specifc_hashtag_in_ten_min)==2:
            return_sign_changes_and_amount_change(sentiment_specifc_hashtag_in_ten_min[0], sentiment_specifc_hashtag_in_ten_min[1])   
        else:
            for i in range(len(sentiment_specifc_hashtag_in_ten_min) - 1):
                sign , change = return_sign_changes_and_amount_change(sentiment_specifc_hashtag_in_ten_min[i], sentiment_specifc_hashtag_in_ten_min[i+1])
                if sign == '+':
                    list_tuples_sudden_chages.append((each_key, sign, change, time.time()))
                if (sign) == '-':
                    list_tuples_sudden_chages.append((each_key, sign, change, time.time()))
                else:
                    continue
    return list_tuples_sudden_chages



In [22]:
import sys
import time

list_keep_ten_min = [] 

class Listener(tweepy.streaming.StreamListener): 
    def __init__(self, output_file=sys.stdout):
        super(Listener,self).__init__()
        self.output_file = output_file
        self.initial_time = time.time()
        self.a_window_text_tweet_list = []
        self.number_tweets = 0
        self.avg_sentiment = 0
        
        self.dict_hashtags = {}
        self.most_popular_hashtag = 'none'
        self.list_tweets_with_max_hashtag = []
        self.most_popular_hashtag_avg_sentiment = 0
         
    def on_status(self, tweet):
        text_tweet = tweet.text
        self.a_window_text_tweet_list.append(text_tweet)
        self.current_time = time.time()
        if self.current_time - self.initial_time >= 60:
            print("summary")
            self.number_tweets = len(self.a_window_text_tweet_list)
            temp_avg = 0
            for a_tweet_txt in self.a_window_text_tweet_list:
                temp_avg += get_average_sentiment_score(a_tweet_txt)
                
            self.avg_sentiment = temp_avg  
                
            print("a_window_text_tweet_list: ", self.a_window_text_tweet_list)
            print("number of tweets in one window: ", self.number_tweets)
            print("avg_sentiment", self.avg_sentiment)
            
            dict_hastags_in_a_window = create_dict_from_hashtags(self.a_window_text_tweet_list)
            self.dict_hashtags = dict_hastags_in_a_window
            
            most_popular_hashtag_in_a_window = find_the_most_popular_hashtag(self.dict_hashtags)
            self.most_popular_hashtag = most_popular_hashtag_in_a_window
            
            list_tweets_with_max_hashtag_in_a_window = return_a_list_tweets_with_max_hasthags(self.a_window_text_tweet_list, self.most_popular_hashtag)
            self.list_tweets_with_max_hashtag = list_tweets_with_max_hashtag_in_a_window
            
            temporary_avg = 0
            for each_tweet_text in self.list_tweets_with_max_hashtag:
                temporary_avg += get_average_sentiment_score(each_tweet_text)
                
            self.most_popular_hashtag_avg_sentiment = temporary_avg
            
            print("dict_hastags_in_a_window: ", self.dict_hashtags)
            print("most_popular_hashtag_in_a_window: ", self.most_popular_hashtag)
            print("list_tweets_with_max_hashtag_in_a_window: ", self.list_tweets_with_max_hashtag)
            print("most_popular_hashtag_avg_sentiment: ", self.most_popular_hashtag_avg_sentiment)

            #*********************************last part
            dict_sentiment_each_hashag_a_window = {}
            for this_hashtag in self.dict_hashtags:
                list_tweets_with_this_hashtag = return_a_list_tweets_with_this_hasthag(self.a_window_text_tweet_list, this_hashtag)
                temp_score_this_hashtag = 0
                for each_text in list_tweets_with_this_hashtag:
                    temp_score_this_hashtag += get_average_sentiment_score(each_text)
                    
                dict_sentiment_each_hashag_a_window[this_hashtag] = temp_score_this_hashtag
            list_keep_ten_min.append(dict_sentiment_each_hashag_a_window)  
                
            #**********************************
            print("Note: This increases in each step--->list_keep_ten_min: ", list_keep_ten_min)
            print("Note: This increases in each step--->len(list_keep_ten_min): ", len(list_keep_ten_min) )
            print("summary end")
            
            self.a_window_text_tweet_list = []
            self.number_tweets = 0
            self.initial_time = self.current_time
            self.avg_sentiment = 0
            
            #####
            self.dict_hashtags = {}
            self.most_popular_hashtag = 'none'
            self.list_tweets_with_max_hashtag = []
            self.most_popular_hashtag_avg_sentiment = 0
            #####
                      
            
    def on_error(self, status_code):
        print(status_code) 
        return False

In [23]:
print(afinn_dict)

{'abandon': -2, 'abandoned': -2, 'abandons': -2, 'abducted': -2, 'abduction': -2, 'abductions': -2, 'abhor': -3, 'abhorred': -3, 'abhorrent': -3, 'abhors': -3, 'abilities': 2, 'ability': 2, 'aboard': 1, 'aborted': -1, 'aborts': -1, 'absentee': -1, 'absentees': -1, 'absolve': 2, 'absolved': 2, 'absolves': 2, 'absolving': 2, 'absorbed': 1, 'abuse': -3, 'abused': -3, 'abuses': -3, 'abusing': -3, 'abusive': -3, 'accept': 1, 'acceptable': 1, 'acceptance': 1, 'accepted': 1, 'accepting': 1, 'accepts': 1, 'accessible': 1, 'accident': -2, 'accidental': -2, 'accidentally': -2, 'accidents': -2, 'acclaim': 2, 'acclaimed': 2, 'accolade': 2, 'accomplish': 2, 'accomplished': 2, 'accomplishes': 2, 'accomplishment': 2, 'accomplishments': 2, 'accusation': -2, 'accusations': -2, 'accuse': -2, 'accused': -2, 'accuses': -2, 'accusing': -2, 'ache': -2, 'achievable': 1, 'aching': -2, 'acquit': 2, 'acquits': 2, 'acquitted': 2, 'acquitting': 2, 'acrimonious': -3, 'active': 1, 'adequate': 1, 'admire': 3, 'admir

# ToDo part #1 (some results are above, e.g. 5000 distinct tweeter.)

In [22]:
listener = Listener()
stream = tweepy.Stream(auth=API.auth, listener=listener) 
print ('Starting!!!!!!!!!!!!!!!!!!') #stream.filter(follow=[nyt.id_str])
try:
    print('Start streaming.') 
    stream.filter(follow=tweeters) #stream.sample(languages=['en'])
except KeyboardInterrupt:
    print("Stopped.")
finally: 
    stream.disconnect() 
    print('Done.')

Starting!!!!!!!!!!!!!!!!!!
Start streaming.
summary
a_window_text_tweet_list:  ['RT @nytimes: In their impeachment defense of President Trump, House Republicans will argue that his decision to withhold military aid from…', 'RT @nytimes: We asked service members, veterans and civilians who have observed Thanksgiving in a conflict zone to share their photographs…', 'RT @nytimes: In their impeachment defense of President Trump, House Republicans will argue that his decision to withhold military aid from…', 'RT @nytimes: Prom Queen, Most Likely to Succeed, Most Talkative: They were in high school when painkillers hit their small Ohio town two de…', '@nytimes You cant be trusted. MAGA', '@nytimes So, they were TOO corrupt to receive the aid, but NOT too corrupt to conduct multiple investigations invol… https://t.co/CWlAbpkmx0', '@nytimes Yawn', '@nytimes Nothing but show business and fake news joining forces to produce this hoax. Wizard of Oz-like.']
number of tweets in one window:  8
avg_s

# ToDo part #2 

In [27]:
listener_two = Listener()
stream_two = tweepy.Stream(auth=API.auth, listener=listener_two) 
print ('Starting!!!!!!!!!!!!!!!!!!')
try:
    print('Start streaming.') 
    stream_two.filter(follow=ids_str)
except KeyboardInterrupt:
    print("Stopped.")
finally: 
    stream_two.disconnect() 
    print('Done.')

Starting!!!!!!!!!!!!!!!!!!
Start streaming.
summary
a_window_text_tweet_list:  ['#SoulPoetry #Sultry #Share #dailyinspirationalpoetry #SoulInspirations #poetry https://t.co/1V66FNXKdt', '@CaymanLes @FrankFigliuzzi1 How can you say that dimwit?', '@TheLastRefuge2 Learn all about "The Trayvon Hoax: Unmasking the Witness Fraud that Divided America" 👉… https://t.co/n6Uub86I0H', '@Rentfreebheasts @Sunkway_China https://t.co/8cMW4q7LUX', '@GlodeJo07 Exacto la hacen menos pero deben estar sorprendidos y que las voluntades y amor a la patria no se compran', "RT @FakeFirstLady44: Joan Rivers calls Obama gay &amp; Michelle a tranny... She's not joking! https://t.co/qXFHUnljyL", '@BlogdoNoblat É assédio sexual que chama?', 'RT @Mzavalagc: Quienes fuimos a la marcha sabemos que mienten https://t.co/uR4CEllV3W', 'LOVE starts with a SMILE , grows with a KISS, and ends with a TEAR\nيبدأ الحب بابتسامة ، وينمو بقبلة ، و ينتهي بدمعة https://t.co/VpGXAALStN', 'RT @peacebeyondme: YOU ASKED FOR IT, WE GOT 

summary
a_window_text_tweet_list:  ['RT @TheDailyEdge: BREAKING: McGahn must testify, says judge. Further delay: “would unquestionably harm the ongoing investigation that the J…', 'RT @kylegriffin1: A reminder regarding the new WaPo report: "The inspector general operates independently of Justice Department leadership,…', 'RT @BNightengale: The #Brewers non-tender Travis Shaw along with Tyler Saladino, Alex Claudio, Junior Guerra and Jimmy Nelson.', 'RT @YAlsarkhi: اگر برخی معترضان #ماهشهر مسلح بودند، اگر شهر خود را در برابر یورش دوشکا و تانک حمایت می\u200cکردند، اما می\u200cتوانید ثابت کنید که حت…', 'RT @RVAwonk: Zuckerberg is defending his decision to let politicians buy space to lie on Facebook, saying: “I think people should be able t…', 'RT @FakeFirstLady44: Have Malia &amp; Sasha Obama gone into complete hiding? Any media interviews or TV shows? Oprah? Ellen? Anything? At ages…', '#SoulPoetry #Sultry #Share #dailyinspirationalpoetry #SoulInspirations #poetry https://t.co/v0qmvyEU

summary
a_window_text_tweet_list:  ['https://t.co/k8uUFwfrQg', 'RT @matthewfsmith: On #GivingTuesday, please stand up for human rights. We need your help and every bit counts. You can make your gift to @…', '@qxa8zprrYwks5CR @Samia1aKmal @KSA24 علي الاقل هو انتكس ايام ما كان الاحتلال بيرفض ان تكون في قوي غيره علي الارض في… https://t.co/wt4uYwSXCC', 'RT @RodrigoGarciaMX: Lo que Alex Lora quiso decir:\n\nEs un privilegio y un honor que el Presidente, en lugar de atender los problemas de ins…', 'RT @nytimes: Additional charges are likely in the criminal case against 2 longtime associates of President Trump’s personal lawyer, Rudy Gi…', '#من_شورشی_هستم🔥 #براندازم🔥 #براندازان٩٧_٩٨ 🔥#شهرهاى_شورشى🔥 #شهرهاى_قيام🔥#هموطن_بپاخيز🔥#اعتراضات_سراسرى 🔥… https://t.co/dx6uYECTp5', 'Por isso brasileiro é assim. Graças a Deus que não sofremos sem rir antes', 'RT @RepAdamSchiff: In addition to fulfilling our Constitutional obligation to conduct oversight of the Executive Branch,\n\nHouse Democrats h…', 'RT

summary
a_window_text_tweet_list:  ["@JeffreeStar I'm raising money for Help. Click to Donate https://t.co/yy60NfTOfb", 'Vejo o vídeo e só penso em putaria ✌✌✌\nMenino tóxico do caralho\nEu que lute', '@DailyMailUK We have some questions for #GhislaineMaxwell', 'RT @bylinetd: Endlessly waiting for stuff that doesn’t happen silently crushes your happily ever after.', 'RT @nytimes: Additional charges are likely in the criminal case against 2 longtime associates of President Trump’s personal lawyer, Rudy Gi…', '@MoveOn @RepBrianFitz @HouseGOP Learn all about "The Trayvon Hoax: Unmasking the Witness Fraud that Divided America… https://t.co/ySn6VjGnCk', "RT @sunnydollkim: I've actively avoided twitter throughout my social media use. But I opened the account because I believe in #HumanityFirs…", 'RT @Mk3Fernando: https://t.co/k8uUFwfrQg', '@RashaAlAqeedi لا مستحيل ههههههه', 'RT @Mk3Fernando: https://t.co/u66W5gmD7y', 'आप सिर्फ चरवाहा विद्यालय के ही बारे में जानते है \nलेकिन ये नही नही जानते 

# Part 4 of section #2

In [28]:
list_dict_hashtags_got_from_twenty_min_run = [{'#SoulPoetry': 0.0, '#Sultry': 0.0, '#Share': 0.0, '#dailyinspirationalpoetry': 0.0, '#SoulInspirations': 0.0, '#poetry': 0.0, '#PALMTREES,': 0.0}, {'#Schiffshow': -2.0, '#SoulPoetry': 0.0, '#Sultry': 0.0, '#Share': 0.0, '#dailyinspirationalpoetry': 0.0, '#SoulInspirations': 0.0, '#poetry': 0.0}, {'#SoulPoetry': 0.0, '#Sultry': 0.0, '#Share': 0.0, '#dailyinspirationalpoetry': 0.0, '#SoulInspirations': 0.0, '#poetry': 0.0}, {'#Brewers': 0.0, '#ماهشهر': 0.0, '#SoulPoetry': 0.0, '#Sultry': 0.0, '#Share': 0.0, '#dailyinspirationalpoetry': 0.0, '#SoulInspirations': 0.0, '#poetry': 0.0, '#FakeObamaFamily': 0.0}, {'#SoulPoetry': 0.0, '#Sultry': 0.0, '#Share': 0.0, '#dailyinspirationalpoetry': 0.0, '#SoulInspirations': 0.0, '#poetry': 0.0, '#DefendersUnion': 1.0, '#W…': 1.0, '#جراحی': 0.0, '#نیزار': 0.0, '#ماهشهر': 0.0}, {'#CottonFest2…': 0.0}, {'#GivingTuesday,': 1.25, '#من_شورشی_هستم🔥': 0.0, '#براندازم🔥': 0.0, '#براندازان٩٧_٩٨': 0.0, '#شهرهاى_قيام🔥#هموطن_بپاخيز🔥#اعتراضات_سراسرى': 0.0}, {'#ماهشهر': 0.0, '#Syria': 0.0}, {}, {'#GhislaineMaxwell': 0.0, '#HumanityFirs…': -0.5, '#جراحی': 0.0, '#نیزار': 0.0, '#ماهشهر': 0.0}, {'#WeAreDefenders': 1.0, '#UAW': 1.0, '#1U': 1.0}]

In [40]:
list_tuples_sudden_chages = return_tuple_regarding_sudden_chages_in_a_particular_hashtag(list_dict_hashtags_got_from_twenty_min_run)

In [43]:
if len(list_tuples_sudden_chages)>0:
    print('alert')
    print(list_tuples_sudden_chages)
else:
    print('no alert')

no alert


In [48]:
list_of_dict_hastag_sentiment = [{'baran':2, 'test': 4}, {'sepid':4}, {'baran':1.5, 'sepid':-3}, {'baran':2}, {'reza':3, 'sepid':2}, {'fire':-2}, {'job':1}, {'president':5}, {'candidate':2}, {'hate':-4}]
list_tuples_sudden_changes = return_tuple_regarding_sudden_chages_in_a_particular_hashtag(list_of_dict_hastag_sentiment)   
if len(list_tuples_sudden_changes)>0:
    print('alert')
    print(list_tuples_sudden_changes)
else:
    print('no alert')

alert
[('baran', '+', 0.3333331111112593, 1575337330.700278), ('sepid', '-', 1.7499995625001092, 1575337330.700282), ('sepid', '-', 1.6666672222224075, 1575337330.7002828)]
